# Overwork Hiding

## Description
Work is performed by agents outside their working hours, but this is not due to
them taking on more work than they should have, but rather their inability.

In [72]:
import pandas as pd
import numpy as np
import altair as alt

In [73]:
log_name = 'propr'

fn_logs = {
    'bpic15': '../data/preproc/bpic2015_disco.csv',
    'bpic17': '../data/preproc/bpic2017.csv',
    'propr': '../data/preproc/proprietary.csv'
}

In [74]:
log = pd.read_csv(fn_logs[log_name])
log

,activity,case_id,start timestamp,complete timestamp,resource,Weekday,"School holidays; 0 = no, 1 = yes",Approval; 1 = low sum to 4 = high sum,Type of loan; 0 = normal; 1 = special,Cost
0,Approval_Branch,005141EEB1240B31C12577DF004F6A77,18/11/2010 15:27,18/11/2010 15:58,004-9-1,Thursday,0.0,2.0,0.0,"0,87 € per minute"
1,Precheck,005141EEB1240B31C12577DF004F6A77,19/11/2010 12:45,19/11/2010 12:46,000-3-01,Friday,0.0,2.0,0.0,"0,87 € per minute"
2,Precheck,005141EEB1240B31C12577DF004F6A77,24/11/2010 8:18,24/11/2010 8:26,000-2-01,Wednesday,0.0,2.0,0.0,"0,87 € per minute"
3,Check_of_Processing_Applications,005141EEB1240B31C12577DF004F6A77,24/11/2010 10:35,24/11/2010 10:35,000-2-01,Wednesday,0.0,2.0,0.0,"0,87 € per minute"
4,Processing_of_Applications,005141EEB1240B31C12577DF004F6A77,2/12/2010 10:46,2/12/2010 10:46,010-23-11,Thursday,0.0,2.0,0.0,"1,02 € per minute"
...,...,...,...,...,...,...,...,...,...,...
18440,Precheck,FFFF329EF772D73CC12577EA00534A1C,13/12/2010 11:38,13/12/2010 11:40,000-2-01,Monday,0.0,2.0,0.0,"0,87 € per minute"
18441,Check_of_Processing_Applications,FFFF329EF772D73CC12577EA00534A1C,13/12/2010 12:06,13/12/2010 12:14,000-2-01,Monday,0.0,2.0,0.0,"0,87 € per minute"
18442,Processing_of_Applications,FFFF329EF772D73CC12577EA00534A1C,22/12/2010 11:48,22/12/2010 11:48,010-23-07,Wednesday,1.0,2.0,0.0,"1,02 € per minute"
18443,Archieving,FFFF329EF772D73CC12577EA00534A1C,22/12/2010 11:49,22/12/2010 13:19,010-23-07,Wednesday,1.0,2.0,0.0,"1,02 € per minute"


## Performance-OH-1

### Identify events indicating activity completion outside working hours

In [75]:
# Input: time ranges indicating the official working hours
# NOTE: for the proprietary dataset, there is already relevant information
# recorded as an event attribute

if log_name == 'propr':
    print(log['Weekday'].unique())
    log['is_work_day'] = log['School holidays; 0 = no, 1 = yes'].apply(
        lambda x: x != 1
    )
    print(log['is_work_day'].unique())
    # NOTE: all events have the same start and complete date in this dataset
    # log['start timestamp'] = pd.to_datetime(log['start timestamp'])
    log['complete timestamp'] = pd.to_datetime(log['complete timestamp'])
    # log['start date'] = log['start timestamp'].dt.date
    # log['complete date'] = log['complete timestamp'].dt.date
    # print(log[log['start date'] != log['complete date']])
    log['complete hour'] = log['complete timestamp'].dt.hour
    log['is_work_hour'] = log['complete hour'].apply(
        lambda x: 9 <= x and x <= 17
    )
    log['is_overwork'] = log.apply(
        lambda row: not(row['is_work_day'] and row['is_work_hour']),
        axis=1
    )
log


['Thursday' 'Friday' 'Wednesday' 'Monday' 'Tuesday']
[ True False]


C:\Users\yangj29\AppData\Local\Temp\ipykernel_1644\2499055932.py:13: UserWarning: Parsing dates in %d/%m/%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  log['complete timestamp'] = pd.to_datetime(log['complete timestamp'])


,activity,case_id,start timestamp,complete timestamp,resource,Weekday,"School holidays; 0 = no, 1 = yes",Approval; 1 = low sum to 4 = high sum,Type of loan; 0 = normal; 1 = special,Cost,is_work_day,complete hour,is_work_hour,is_overwork
0,Approval_Branch,005141EEB1240B31C12577DF004F6A77,18/11/2010 15:27,2010-11-18 15:58:00,004-9-1,Thursday,0.0,2.0,0.0,"0,87 € per minute",True,15,True,False
1,Precheck,005141EEB1240B31C12577DF004F6A77,19/11/2010 12:45,2010-11-19 12:46:00,000-3-01,Friday,0.0,2.0,0.0,"0,87 € per minute",True,12,True,False
2,Precheck,005141EEB1240B31C12577DF004F6A77,24/11/2010 8:18,2010-11-24 08:26:00,000-2-01,Wednesday,0.0,2.0,0.0,"0,87 € per minute",True,8,False,True
3,Check_of_Processing_Applications,005141EEB1240B31C12577DF004F6A77,24/11/2010 10:35,2010-11-24 10:35:00,000-2-01,Wednesday,0.0,2.0,0.0,"0,87 € per minute",True,10,True,False
4,Processing_of_Applications,005141EEB1240B31C12577DF004F6A77,2/12/2010 10:46,2010-12-02 10:46:00,010-23-11,Thursday,0.0,2.0,0.0,"1,02 € per minute",True,10,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18440,Precheck,FFFF329EF772D73CC12577EA00534A1C,13/12/2010 11:38,2010-12-13 11:40:00,000-2-01,Monday,0.0,2.0,0.0,"0,87 € per minute",True,11,True,False
18441,Check_of_Processing_Applications,FFFF329EF772D73CC12577EA00534A1C,13/12/2010 12:06,2010-12-13 12:14:00,000-2-01,Monday,0.0,2.0,0.0,"0,87 € per minute",True,12,True,False
18442,Processing_of_Applications,FFFF329EF772D73CC12577EA00534A1C,22/12/2010 11:48,2010-12-22 11:48:00,010-23-07,Wednesday,1.0,2.0,0.0,"1,02 € per minute",False,11,True,True
18443,Archieving,FFFF329EF772D73CC12577EA00534A1C,22/12/2010 11:49,2010-12-22 13:19:00,010-23-07,Wednesday,1.0,2.0,0.0,"1,02 € per minute",False,13,True,True


### Compute the frequency by events and cases performed by each resource

In [76]:
# by events
df_gb_events = log.groupby(['resource']).agg(
    num_ow_events=pd.NamedAgg(column='is_overwork', aggfunc=(
        lambda s: sum(s == True)
    )),
    num_events=pd.NamedAgg(column='is_overwork', aggfunc=len),
).reset_index()
df_gb_events['freq_ow_events'] = df_gb_events['num_ow_events'] / df_gb_events['num_events'] * 100
# df_gb_events

In [77]:
alt.hconcat(
    alt.Chart(
        data=df_gb_events, 
        title='Distribution of overwork events'
    ).mark_boxplot().encode(
        y=alt.Y('freq_ow_events:Q')
            .title('Frequency of overwork events by resources')
    ),
    alt.Chart(
        data=df_gb_events,
        title='Relative frequency of overwork events'
    ).transform_density(
        'freq_ow_events',
        as_=['freq_ow_events', 'density']
    ).mark_area().encode(
        x=alt.X('freq_ow_events:Q')
            .title('Frequency of overwork events by resources'),
        y='density:Q'
    )
)

alt.HConcatChart(...)

### Identify resources with significantly higher frequency of overwork


In [91]:
# Determine the threshold
# resources with relative frequency of overwork higher than threshold
# are identified as significantly higher frequency of overwork

threshold = df_gb_events['freq_ow_events'].quantile(0.75)
df_gb_events['is_identified'] = False
df_gb_events.loc[
    df_gb_events['freq_ow_events'] > threshold, 'is_identified'
] = True

In [94]:
alt.layer(
    alt.Chart(
        data=df_gb_events, 
        title='Relative frequency of overwork events by resources'
    ).mark_point().encode(
        x=alt.X('resource:O'),
        y=alt.Y('freq_ow_events:Q').title(
            'Frequency of overwork events by resources'
        ),
        color=alt.condition(
            alt.datum.is_identified,
            alt.value('red'), alt.value('blue')
        ),
        size=alt.Size('num_events:Q'),
        tooltip=['resource', 'num_events', 'num_ow_events', 'freq_ow_events']
    ),
    alt.Chart().mark_rule(color='red').encode(
        y=alt.Y(datum=threshold)
    )
)

alt.LayerChart(...)